In [6]:
from ortools.linear_solver import pywraplp
import re
import pandas as pd 

In [2]:
solver = pywraplp.Solver('Disruption_Sales',pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

# Data

In [5]:
with open('data_1.txt', 'r') as fp:
    x1 = fp.readlines()

with open('data_2.txt', 'r') as fp:
    x2 = fp.readlines()

x1 = [re.sub('\n','',i) for i in x1]
x2 = [re.sub('\n','',i) for i in x2]

In [7]:
df = pd.DataFrame([tuple(re.split(',',i)) for i in x1])
df.columns = df.iloc[0]
df.drop(df.index[0], inplace=True)
df.reset_index(drop=True, inplace=True)

df_2 = pd.DataFrame([tuple(re.split(',',i)) for i in x2])
df_2.columns = df_2.iloc[0]
df_2.drop(df_2.index[0], inplace=True)
df_2.reset_index(drop=True, inplace=True)

In [8]:
df[' Inventory Cost(100 units)']=df[' Inventory Cost(100 units)'].astype(int)

In [9]:
d = {}
for i in range(8):
    item = df_2['ItemID'].iloc[i]
    cfg1 = df_2[' CFG1'].iloc[i] 
    cfg2 = df_2[' CFG2'].iloc[i]
    cfg3 = df_2[' CFG3'].iloc[i]
    cfg4 = df_2[' CFG4'].iloc[i]
    d[(item,'CFG1')] = int(cfg1)
    d[(item,'CFG2')] = int(cfg2)
    d[(item,'CFG3')] = int(cfg3)
    d[(item,'CFG4')] = int(cfg4)
B = d

# Parameters

In [59]:
# Create a list of the Plant Nodes
M = df['Plant'].tolist()
M1 = df['Plant'].tolist()[0:8]
M2 = df['Plant'].tolist()[8:]
N = [f'C{i}' for i in range(10)]

# Create a dictionary of capacity for all Plant nodes
capacity = dict(zip(M1,df[' Capacity']))

# Create a dictionary of disruption probability for all Plant nodes
dis_prob = dict(zip(M2,df[' Disruption Probability(1 - µ)']))

# Create a dictionary of inventory costs
inv_cost_x = dict(zip(M1,df[' Inventory Cost(100 units)'].iloc[0:8]))
inv_cost_y = dict(zip(M2,df[' Inventory Cost(100 units)'].iloc[8:]))


# Demand from all customers for different products
demand = {
    'C0':{'CFG1':2, 'CFG2':1, 'CFG3':0, 'CFG4':0},
    'C1':{'CFG1':2, 'CFG2':0, 'CFG3':2, 'CFG4':4},
    'C2':{'CFG1':1, 'CFG2':5, 'CFG3':0, 'CFG4':0},
    'C3':{'CFG1':1, 'CFG2':1, 'CFG3':1, 'CFG4':0},
    'C4':{'CFG1':6, 'CFG2':0, 'CFG3':2, 'CFG4':2},
    'C5':{'CFG1':4, 'CFG2':0, 'CFG3':0, 'CFG4':3},
    'C6':{'CFG1':4, 'CFG2':1, 'CFG3':2, 'CFG4':1},
    'C7':{'CFG1':2, 'CFG2':0, 'CFG3':1, 'CFG4':0},
    'C8':{'CFG1':1, 'CFG2':2, 'CFG3':1, 'CFG4':0},
    'C9':{'CFG1':3, 'CFG2':0, 'CFG3':1, 'CFG4':2},
    }

# Lost Sales Penalty
f = {
    'C0':2.7, 'C1':1.7, 'C2':2.3, 'C3':1.5, 
    'C4':2.7, 'C5':1.7, 'C6':2.3, 'C7':1.5, 
    'C8':2.7, 'C9':1.7
    }

# Decision Variables

In [27]:
# Material flow from Stage 1 to Stage 2
x = [[solver.NumVar(0.0, solver.infinity(), f'x_{i}_{j}') for j in (M2)] for i in (M1)]

In [28]:
# Material flow from for Stage 2 to Customer
y = [[solver.NumVar(0.0, solver.infinity(), f'y_{j}_{k}') for k in (N)] for j in (M2)]

In [30]:
# Lost Sales
l = [solver.NumVar(0.0, solver.infinity(), f'lost_sales_{k}') for k in (N)]

In [24]:
# Goods Produced
u = [solver.NumVar(0.0, solver.infinity(), f'good_produced_{j}') for j in (M)]

In [25]:
# Inventory
r = [solver.NumVar(0.0, solver.infinity(), f'inventory_{j}') for j in (M)]

# Objective

In [60]:
solver.Minimize(solver.Sum([f[k] * l[k] for k in N]))

TypeError: list indices must be integers or slices, not str

# Constraints

In [26]:
# Define the constraints   
constraint_list=[]

\begin{equation*}
\left( \sum_{i=1}^{M1} x_{ij} \right) \geq \left( \sum_{k=1}^N y_{jk} \right)..... \text{for all j}
\end{equation*}

In [39]:
for j in range(len(M2)):
    solver.Add(solver.Sum([x[i, j] for i in range(len(M1))]) >= solver.Sum([y[j, k] for k in range(len(N))]))

TypeError: list indices must be integers or slices, not tuple

\begin{equation*}
\left( d_{k} \right) \leq \left( \sum_{j=1}^{M2} y_{jk} \right) + \left( l_{k} \right).....\text{for all k}
\end{equation*}

In [38]:
for i in range(len(M2)):
    print(i)

0
1
2
3
